## 콘텐츠 기반 필터링
- 데이터(크롤링) : news 텍스트(자연어) 데이터
- 토큰분석기 : Mecab - Doc2Vec 모델 사용 / TF2IDF, 

### 방법
- 1. 뉴스 기사를 토큰 분석기로 토큰화 함 (뉴욕, 연합뉴스, 윤영숙, 연합)
- 2. Mecab의 Doc2Vec 모델로 해당 Token들을 학습함 (document를 vector로)
  - input : [토큰화 된 단어들] document / output : vector[1.0, 1.5, 3.6]
- 3. '내 데이터'를 모델에 넣고 돌림 
  - input : [토큰화 된 단어들] document / output : vector
- 4. 가장 유사한 top 10개 데이터 뽑음.


### 정리
- 학습 데이터들을 Token화, vector화를 거쳐 model에 저장
- 유저는 신규 텍스트 데이터를 model에 넣어 가장 유사한 문장을 뽑음.

- ex1) 모델 학습 : 데이터 형태 : ([맥주 리뷰 token들], [맥주이름])
  - 맥주 리뷰 document들이 학습 된 상태, '내가 먹고 싶은 맥주는?' 질문에 '청량하고 시원하고 존맛탱' 하고 넣으면 가장 유사한 document를 뽑아고, 유사도가 가장 높은 데이터의 code 컬럼(하이네켄) 추천


In [ ]:
import pandas as pd

df = pd.read_csv('news_data.csv')
print(df.shape)
df.head(3)

(300, 2)


,news,code
0,(뉴욕=연합뉴스) 윤영숙 연합인포맥스 특파원 = 뉴욕증시는 연방준비제도(연준·Fed...,경제
1,박종석 정신과 전문의 /고운호 기자 박종석 정신과 전문의 /고운호 기자\n\n주식으...,경제
2,미국 뉴욕증권거래소(NYSE). © AFP=뉴스1 자료 사진 미국 뉴욕증권거래소(N...,경제
3,25일(현지시간) 독일 뒤스부르크 항구에서 선적을 기다리는 수출 차량들.AP뉴시스 ...,경제
4,뉴욕증시가 하락 출발했다.\n\n\n\n25일(현지시간) 오전 9시35분 현재 뉴욕...,경제


In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd 'Mecab-ko-for-Google-Colab'
!bash install_mecab-ko_on_colab_light_220429.sh

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

df['token'] = pd.Series(map(lambda x: mecab.nouns(x), df['news']))
df['token']

0      [뉴욕, 연합뉴스, 윤영숙, 연합, 인포, 맥스, 특파원, 뉴욕, 증시, 연방, 준...
1      [박종석, 정신, 전문의, 호, 기자, 박종석, 정신, 전문의, 호, 기자, 주식,...
2      [미국, 뉴욕, 증권거래소, 뉴스, 자료, 사진, 미국, 뉴욕, 증권거래소, 뉴스,...
3      [일, 현지, 시간, 독일, 뒤스부르크, 항구, 선적, 수출, 차량, 뉴시스, 일,...
4      [뉴욕, 증시, 하락, 출발, 일, 현지, 시간, 오전, 시, 분, 뉴욕, 증시, ...
                             ...                        
295    [짬, 회원, 명, 산문, 집, 고양시, 문학, 공부, 반, 경기, 고양시, 문학,...
296    [우울증, 사람, 코로나, 백, 관련, 거짓, 정보, 연구, 결과, 사진, 클립아트...
297    [기상청, 일, 전국, 구름, 날씨, 것, 예보, 사진, 기상청, 기상청, 일, 전...
298    [스포츠, 경향, 년, 진실, 사랑, 번, 만남, 백, 번, 만남, 것, 년, 사공...
299    [일, 오후, 광주, 북구, 보건소, 선별, 진료소, 코로나, 검사, 시민, 대기,...
Name: token, Length: 300, dtype: object

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['token'])] # 각 문장별 word
print(documents)
model = Doc2Vec(documents, vector_size=100, window=3, epochs=10, min_count=0, workers=4)

# 100번쨰 행 vector 화
infered_doc_vec = model.infer_vector(df['token'][10])

# 가장 유사한 문서 10개만 뽑아보자.
most_similar_docs = model.docvecs.most_similar([infered_doc_vec], topn=10)

for index, similarity in most_similar_docs:
  print(index, 'similarity: ', similarity, documents[index])

[TaggedDocument(words=['뉴욕', '연합뉴스', '윤영숙', '연합', '인포', '맥스', '특파원', '뉴욕', '증시', '연방', '준비', '제도', '연준', '연방', '공개', '시장', '위원회', '정례', '회의', '하락', '일', '미', '동부', '시간', '오전', '시', '분', '뉴욕', '증권거래소', '다우존스', '산업', '평균', '지수', '전장', '포인트', '하락', '스탠더드', '앤드', '푸', '어스', '지수', '전장', '인트', '기술', '주', '중심', '나스닥', '지수', '전장', '포인트', '기록', '투자자', '이날', '시작', '다음', '날', '종료', '정례회', '결과', '주목', '연준', '이번', '회의', '월', '금리', '인상', '신호', '것', '예상', '시장', '금리', '인상', '신호', '대차', '대조표', '축소', '양', '긴축', '연준', '계획', '주목', '전날', '다우', '지수', '연준', '긴축', '우려', '우크라이나', '지정학', '우려', '변동성', '확대', '장중', '천', '포인트', '이상', '하락', '나스닥', '지수', '지수', '상승', '반전', '변동성', '결과', '확인', '때', '지속', '가능', '월가', '공포', '지수', '변동성', '지수', '기록', '중', '우크라이나', '긴장', '계속', '전날', '로이드', '오스틴', '미', '국방', '장관', '미군', '천', '명', '유럽', '배치', '준비', '태세', '강화', '명령', '나토', '필요', '경우', '해당', '미군', '병력', '대부분', '나토', '속대', '응군', '참여', '나토', '소속', '유럽', '국가', '만일', '사태', '대비', '동유럽', '추가', '병력', '자원', '것', '검토', '조', '바이든', '미국', 

## User Based Coolaborate Filtering
- movie data ,rating data

In [ ]:
import os
os.getcwd()

# %cd /content

/content


In [ ]:
# %cd '../'
# %cd .
# /content/ratings.csv
import pandas as pd
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 300)

movie_ratings = pd.merge(ratings, movies, on='movieId')
print(movie_ratings.shape)
movie_ratings.head(3)

(100836, 6)


,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [ ]:
# pivot table - userid / titl
title_user = movie_ratings.pivot_table('rating', index='userId', columns='title')
title_user = title_user.fillna(0) # 점수 매긴 기록 없으면 0으로
title_user.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),...,eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0


In [ ]:
# cosine 유사도 측정
from sklearn.metrics.pairwise import cosine_similarity
user_based_collab = pd.DataFrame(cosine_similarity(title_user, title_user), index=title_user.index, columns=title_user.index)
print(user_based_collab.shape)
user_based_collab.head(3)

(610, 610)


userId,1,2,3,4,5,...,606,607,608,609,610
userId,,,,,,,,,,,
1,1.000000,0.027283,0.05972,0.194395,0.129080,...,0.164191,0.269389,0.291097,0.093572,0.145321
2,0.027283,1.000000,0.00000,0.003726,0.016614,...,0.028429,0.012948,0.046211,0.027565,0.102427
3,0.059720,0.000000,1.00000,0.002251,0.005020,...,0.012993,0.019247,0.021128,0.000000,0.032119


In [ ]:
# 만약 해당 유저가 아직 보지 않은 영화에 대해서, 평점을 예측하고자 한다면?
# (어떤 유저와 비슷한 정도 * 그 유저가 영화에 대해 부여한 평점) 을 더해서 (유저와 비슷한 정도의 합)으로 나눠보면 됨!
# index_list 는 비슷한 유저의 id 값 리스트 / weight_list 는 비슷한 유저와의 유사도 리스트
user_index_list = user_based_collab[1].sort_values(ascending=False)[:10].index.tolist()
user_weight_list = user_based_collab[1].sort_values(ascending=False)[:10].tolist()

In [ ]:
# 1번 유저가 다크나이트를 보고 어떤 평점을 부여할지 예측
movie_title = 'Dark Knight, The (2008)'
weighted_sum, weighted_user = [], []
for i in range(1, 10):
    # 해당 영화를 보고 평점을 부여한 사람들의 유사도와 평점만 추가 (즉, 0이 아닌 경우에만 계산에 활용)
    if int(title_user[movie_title][user_index_list[i]]) is not 0:
        weighted_sum.append(title_user[movie_title][user_index_list[i]] * user_weight_list[i]) # 평점 * 유사도 추가
        weighted_user.append(user_weight_list[i]) # 유사도 추가

print(weighted_sum, weighted_user)
# 총 평점*유사도 / 총 유사도를 토대로 평점 예측
print(sum(weighted_sum) / sum(weighted_user))

[1.48364788177318]
[0.3296995292829289]
4.5


In [ ]:
# 가장 유사한 유저 찾기
user_num = user_based_collab[1].sort_values(ascending=False)[:10].index[1] # 266, numpy.int64
result = title_user.query(f"userId == {user_num}").sort_values(ascending=False, by=user_num, axis=1)
result

title,Evil Dead II (Dead by Dawn) (1987),Dirty Work (1998),Goodfellas (1990),"Fish Called Wanda, A (1988)","Fisher King, The (1991)",...,Fugitives (1986),Full Moon in Blue Water (1988),Fullmetal Alchemist 2018 (2017),Fullmetal Alchemist the Movie: Conqueror of Shamballa (Gekijô-ban hagane no renkinjutsushi: Shanbara wo yuku mono) (2005),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,
266,5.0,5.0,5.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0


## Item Based Coolaborate Filtering

In [ ]:
# 이번에는 Index에 title이 들어감!
user_title = movie_ratings.pivot_table('rating', index='title', columns='userId')

user_title = user_title.fillna(0)
print(user_title)

userId                                     1    2    3    4    5    ...  606  607  608  609  610
title                                                               ...                         
'71 (2014)                                 0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  4.0
'Hellboy': The Seeds of Creation (2004)    0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0
'Round Midnight (1986)                     0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0
'Salem's Lot (2004)                        0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0
'Til There Was You (1997)                  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0
...                                        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
eXistenZ (1999)                            0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  4.5  0.0  0.0
xXx (2002)                                 0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  3.5  0.0  2.0
xXx: State of the Union (2005)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

item_based_collab = cosine_similarity(user_title, user_title)
print(item_based_collab)

In [ ]:
item_based_collab = pd.DataFrame(item_based_collab, index=user_title.index, columns=user_title.index)

In [ ]:
print(item_based_collab)

In [ ]:
# 다크나이트와 비슷하게 유저들로부터 평점을 부여받은 영화들은?
print(item_based_collab['Dark Knight, The (2008)'].sort_values(ascending=False)[:10])

## 잠재 요인 협업 필터링

In [ ]:
!pip install scikit-surprise

In [ ]:
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split

reader = Reader(rating_scale = (1.0, 5.0))
data = Dataset.load_from_df(df=ratings[['userId', 'movieId', 'rating']], reader=reader, )

In [ ]:
# train-test-split
train, test = train_test_split(data, test_size=0.2, shuffle=True, random_state=2022)

# model train
algo = SVD(n_factors=50, n_epochs=20, random_state=2022) # n_factors = 잠재 요인 크기 (파라미터 갯수)
algo.fit(train)

In [ ]:
pred = algo.predict('1','1') # r_ui는 실제로 유저가 부여한 평점
pred

Prediction(uid='1', iid='1', r_ui=None, est=3.500712798135568, details={'was_impossible': False})

In [ ]:
# ratings
# userId	movieId	rating
# 1        	1	      4.0

In [ ]:
movies[movies['movieId']==210]

,movieId,title,genres
178,210,Wild Bill (1995),Western


In [ ]:
movies[movies['title']=='Wild Bill (1995)']

,movieId,title,genres
178,210,Wild Bill (1995),Western
